In [1]:
import openpyxl
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import accuracy_score
from numpy.linalg import norm
from sklearn.model_selection import LeaveOneOut
from decimal import *
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import math
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

number_runnings = 50

hepatitis = pd.read_csv("hepatitis.csv")
hepatitis['ALB'] = hepatitis['ALB'].replace(to_replace=np.nan,value = hepatitis['ALB'].median())
hepatitis['ALP'] = hepatitis['ALP'].replace(to_replace=np.nan,value = hepatitis['ALP'].median())
hepatitis['ALT'] = hepatitis['ALT'].replace(to_replace=np.nan,value = hepatitis['ALT'].median())
hepatitis['AST'] = hepatitis['AST'].replace(to_replace=np.nan,value = hepatitis['AST'].median())
hepatitis['BIL'] = hepatitis['BIL'].replace(to_replace=np.nan,value = hepatitis['BIL'].median())
hepatitis['CHE'] = hepatitis['CHE'].replace(to_replace=np.nan,value = hepatitis['CHE'].median())
hepatitis['CHOL'] = hepatitis['CHOL'].replace(to_replace=np.nan,value = hepatitis['CHOL'].median())
hepatitis['CREA'] = hepatitis['CREA'].replace(to_replace=np.nan,value = hepatitis['CREA'].median())
hepatitis['GGT'] = hepatitis['GGT'].replace(to_replace=np.nan,value = hepatitis['GGT'].median())
hepatitis['PROT'] = hepatitis['PROT'].replace(to_replace=np.nan,value = hepatitis['PROT'].median())

hepatitis['Outcome'].value_counts()
Y = hepatitis['Outcome']
target_names=["0", "1"]
noOfInstances = hepatitis.shape[0]
df = pd.DataFrame(hepatitis)
corr = df.corrwith(df["Outcome"])
print(corr)

X = hepatitis[['Age',	'Sex',	'ALB',	'ALP',	'ALT',	'AST',	'BIL',	'CHE',	'CHOL',	'CREA',	'GGT',	'PROT']]
maximums = np.amax(np.array(X), axis=0)
#X = diabetes.iloc[:,:-1].values
X_featurenames = X.columns
counter = Counter(Y)
print(counter[0])
print(counter[1])

feature_names = ['Age',	'Sex',	'ALB',	'ALP',	'ALT',	'AST',	'BIL',	'CHE',	'CHOL',	'CREA',	'GGT',	'PROT']
results = pd.read_excel('resultsSignsHepatitisMPMS_MinMax.xlsx')
noOfIterations = 100

model = XGBClassifier()
# fit the model
model.fit(X, Y)
# get importance
importance = model.feature_importances_
# summarize feature importance
for i in range(len(feature_names)):
    print(importance[i])

df_LIME = pd.read_excel('rankingsHepatitisLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]
W_LIME = []
df_TreeShap = pd.read_excel('rankingsHepatitisTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_KernelShap = pd.read_excel('rankingsHepatitisKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
W_Shap = []
W_all = []

for instance in range (0, noOfInstances):
    m = pd.DataFrame(columns = ['L', 'S'])
    line_LIME  = [results.iloc[instance, 1], results.iloc[instance, 2]]
    m.loc[0] = line_LIME
    line_Shap  = [results.iloc[instance, 3], results.iloc[instance, 4]]
    m.loc[1] = line_Shap
    A = []
    for i in range(0, 2):
        s = 0
        for j in range(0, 2):
            s = s + m.iloc[i, j] + m.iloc[j, i]
        s = s / 4
        A.append(s)
  
    data_LIME = []
    data_Shap = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
        
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
        
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    w_LIME = []
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    w_Shap = []
    
    for i in range(noOfIterations):
        x = [0] * len(feature_names)
        for k in range(0, 2):
            x = x + A[k] * M_LIME.loc[i].values.flatten()
        y = sum(A)
        w = [0] * len(feature_names)
        for k in range(len(feature_names)):
            w[k] = x[k] / y
        w_LIME.append(w)

        x = [0] * len(feature_names)
        for k in range(0, 2):
            x = x + A[k] * M_Shap.loc[i].values.flatten()
        y = sum(A)
        w = [0] * len(feature_names)
        for k in range(len(feature_names)):
            w[k] = x[k] / y
        w_Shap.append(w)
        
    averageLIME = np.mean(w_LIME, axis=0)
    W_LIME.append(averageLIME)
    averageShap = np.mean(w_Shap, axis=0)
    W_Shap.append(averageShap)
    averageMatrix = [averageLIME, averageShap]
    average = np.mean(averageMatrix, axis=0)
    sum_average = sum(average)
    sum_corr = sum(corr) - 1
    for i in range(0, len(feature_names)):
        #average[i] = (average[i] / sum_average + corr.iloc[i] / sum_corr) / 2
        #average[i] = average[i] * corr.iloc[i]
        #average[i] = average[i] * (corr.iloc[i] / sum_corr)
        #average[i] = average[i] + corr.iloc[i]
        average[i] = average[i] 
    W_all.append(average)
    # print(averageLIME)
    # print(averageAnchor)
    # print(averageShap)
    # print(average)
    # print("============")

average_all = np.nanmean(W_all, axis=0)
print(average_all)


def compute_weights(distance):
    weights = []
    for i in range(len(distance)): 
        weights.append(norm(W_all[i]) * distance[i])
    return weights

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
    
# knn = KNeighborsClassifier(n_neighbors = 5)
# knn.fit(X_train,Y_train) 
# Y_pred = knn.predict(X_test)
# print(accuracy_score(Y_test,Y_pred))
# print(f1_score(Y_test,Y_pred))

#X2 = scaler.fit_transform(X)
scale_pos_weight = counter[0] / counter[1]
model = xgb.XGBClassifier(scale_pos_weight = scale_pos_weight)
# model.fit(X_train, Y_train)
# pred = model.predict(X_test)
# accuracy = accuracy_score(Y_test, pred)
# print("Accuracy:", accuracy)
scores = cross_val_score(model, X, Y, cv = 10, scoring='roc_auc')
print(np.mean(scores))


for i in range(noOfInstances):
    for j in range(len(feature_names)):
        X.iloc[i, j] = X.iloc[i, j] * math.sqrt(W_all[i][j])


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle = True)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#X = scaler.fit_transform(X)
#knn = KNeighborsClassifier(n_neighbors = 5, weights = 'uniform')
model = xgb.XGBClassifier(scale_pos_weight = scale_pos_weight)
scores = cross_val_score(model, X, Y, cv = 10, scoring='roc_auc')
print(np.mean(scores))


# knn = KNeighborsClassifier(n_neighbors = 5)
# knn.fit(X_train,Y_train) 
# Y_pred = knn.predict(X_test)
# print(accuracy_score(Y_test,Y_pred))
# print(f1_score(Y_test,Y_pred))
    
# knn = KNeighborsClassifier(n_neighbors = 5, weights = compute_weights)
# knn.fit(X_train,Y_train) 
# Y_pred = knn.predict(X_test)
# print(accuracy_score(Y_test,Y_pred))
# print(f1_score(Y_test,Y_pred))
# print("---------------")

# knn = KNeighborsClassifier(n_neighbors = 5, weights = compute_weights)
# scores = cross_val_score(knn, X, Y, cv=20)
# print(scores)
# print(np.mean(scores))

# cv = LeaveOneOut()
# s = 0
# for train_ix, test_ix in cv.split(X):
#     knn = KNeighborsClassifier(n_neighbors = 5, weights = compute_weights)
#     X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
#     Y_train, Y_test = Y[train_ix], Y[test_ix]
#     knn.fit(X_train, Y_train)  
#     pred = knn.predict(X_test)
#     accuracy_classifier = accuracy_score(Y_test, pred)
#     s = s + accuracy_classifier
# print(s / len(X))

def euclidian_distance(a, b):
    dim = len(a)
    distance = 0
    for i in range(dim):
        distance += abs(a[i] - b[i])**2  
    distance = math.sqrt(distance)    
    return distance


def knn_predict_non_weighted(X_train, X_test, Y_train, Y_test, k):
    
    from collections import Counter
    Y_hat_test = []

    for test_point in X_test:
        distances = []
        for train_point in X_train:
            dim = len(train_point)
            distance = 0
            for i in range(dim):
                distance = distance + math.sqrt((train_point[i] - test_point[i]) * (train_point[i] - test_point[i]))
            distances.append(distance)

        df_dists = pd.DataFrame(data = distances, columns = ['dist'], 
                                index = Y_train.index)

        df_nn = df_dists.sort_values(by=['dist'], axis = 0)[:k]

        counter = Counter(Y_train[df_nn.index])

        prediction = counter.most_common()[0][0]

        Y_hat_test.append(prediction)
        
    return Y_hat_test



def knn_predict_weighted(X_train, X_test, Y_train, Y_test, k):
    
    from collections import Counter
    Y_hat_test = []

    for test_point in X_test:
        distances = []
        for train_point in X_train:
            dim = len(train_point)
            distance = 0
            for i in range(dim):
                distance = distance + math.sqrt(average_all[i] * (train_point[i] - test_point[i]) * (train_point[i] - test_point[i]))
            distances.append(distance)

        df_dists = pd.DataFrame(data = distances, columns = ['dist'], 
                                index = Y_train.index)

        df_nn = df_dists.sort_values(by=['dist'], axis = 0)[:k]

        counter = Counter(Y_train[df_nn.index])

        prediction = counter.most_common()[0][0]

        Y_hat_test.append(prediction)
        
    return Y_hat_test

outputs = pd.DataFrame(columns = ['ACC_NW', 'F1_NW', 'ROC_NW', 'ACC_W', 'F1_W', 'ROC_W'])
acc_non_weighted = 0
acc_weighted = 0
f1_non_weighted = 0
f1_weighted = 0
roc_non_weighted = 0
roc_weighted = 0
for i in range(number_runnings):
    output = []
    #print('------------------')
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, shuffle = True)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    Y_hat_test = knn_predict_non_weighted(X_train, X_test, Y_train, Y_test, k=5)
    #print(f1_score(Y_test, Y_hat_test))
    acc_non_weighted += accuracy_score(Y_test, Y_hat_test)
    output.append(accuracy_score(Y_test, Y_hat_test))
    f1_non_weighted += f1_score(Y_test, Y_hat_test)
    output.append(f1_score(Y_test, Y_hat_test))
    roc_non_weighted += roc_auc_score(Y_test, Y_hat_test)
    output.append(roc_auc_score(Y_test, Y_hat_test))
    
    Y_hat_test = knn_predict_weighted(X_train, X_test, Y_train, Y_test, k=5)
    #print(f1_score(Y_test, Y_hat_test))
    acc_weighted += accuracy_score(Y_test, Y_hat_test)
    output.append(accuracy_score(Y_test, Y_hat_test))
    f1_weighted += f1_score(Y_test, Y_hat_test)
    output.append(f1_score(Y_test, Y_hat_test))
    roc_weighted += roc_auc_score(Y_test, Y_hat_test)
    output.append(roc_auc_score(Y_test, Y_hat_test))
    outputs.loc[i] = output


acc_non_weighted = acc_non_weighted / number_runnings
acc_weighted = acc_weighted / number_runnings
f1_non_weighted = f1_non_weighted / number_runnings
f1_weighted = f1_weighted / number_runnings
roc_non_weighted = roc_non_weighted / number_runnings
roc_weighted = roc_weighted / number_runnings
output = []
output.append(acc_non_weighted)
output.append(f1_non_weighted)
output.append(roc_non_weighted)
output.append(acc_weighted)
output.append(f1_weighted)
output.append(roc_weighted)

outputs.loc[number_runnings] = output

outputs.to_excel("accuraciesHepatitisMPMS_MinMax_" + str(number_runnings) + ".xlsx")

# cv = LeaveOneOut()
# s_non_weighted = 0
# s_weighted = 0
# for train_ix, test_ix in cv.split(X):
#     X_train, X_test = X.to_numpy()[train_ix, :], X.to_numpy()[test_ix, :]
#     Y_train, Y_test = Y[train_ix], Y[test_ix]
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
#     Y_hat_test_non_weighted = knn_predict_non_weighted(X_train, X_test, Y_train, Y_test, k=5)
#     s_non_weighted = s_non_weighted + accuracy_score(Y_test, Y_hat_test_non_weighted)
#     Y_hat_test_weighted = knn_predict_weighted(X_train, X_test, Y_train, Y_test, k=5)
#     s_weighted = s_weighted + accuracy_score(Y_test, Y_hat_test_non_weighted)
# print(s_non_weighted / len(X))
# print(s_weighted / len(X))

Outcome    1.000000
Age        0.037781
Sex       -0.071663
ALB       -0.179609
ALP       -0.067737
ALT        0.087787
AST        0.621724
BIL        0.398451
CHE       -0.230785
CHOL      -0.266003
CREA       0.136772
GGT        0.437680
PROT       0.084082
dtype: float64
540
75
0.0122319255
0.0
0.025757944
0.08343849
0.10376467
0.53502154
0.08025975
0.031697087
0.022222595
0.013443293
0.0726351
0.019527644
[ 4.70849192  3.37552962  4.81063734  7.96972172  8.35816876 10.67863555
  8.15702873  4.36135548  4.78046679  5.92082585  8.47254039  5.90659785]
0.9711309523809524


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_4300\4133123720.py:174: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '70.65408693062278' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.iloc[i, j] = X.iloc[i, j] * math.sqrt(W_all[i][j])


0.9819444444444445
